# Climat Change - Exercise1

In [1]:
import matplotlib.pyplot as plt
import numpy as np

## Load model

In [2]:
import tensorflow_hub as hub
import tensorflow as tf

print("tensorflow: ", tf.version.VERSION)

tensorflow:  2.11.0


In [5]:
model = tf.saved_model.load(PATH_TO_MODEL)

## converting

In [6]:
%timeit -n 1
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model_float16 = converter.convert()
with open(TFLITE_FILE_PATH_FLOAT16, 'wb') as f:
  f.write(tflite_quant_model_float16)


INFO:tensorflow:Assets written to: C:\Users\lukas\AppData\Local\Temp\tmpzg9px4f7\assets


INFO:tensorflow:Assets written to: C:\Users\lukas\AppData\Local\Temp\tmpzg9px4f7\assets


In [25]:
%timeit -n 1
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float32]
tflite_quant_model_float32 = converter.convert()
with open(TFLITE_FILE_PATH_FLOAT32, 'wb') as f:
  f.write(tflite_quant_model_float32)


INFO:tensorflow:Assets written to: C:\Users\lukas\AppData\Local\Temp\tmp559je1cq\assets


INFO:tensorflow:Assets written to: C:\Users\lukas\AppData\Local\Temp\tmp559je1cq\assets


In [4]:
%timeit -n 1
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]
tflite_quant_model_int8 = converter.convert()
with open(TFLITE_FILE_PATH_INT8, 'wb') as f:
  f.write(tflite_quant_model_int8)

ValueError: For full integer quantization, a `representative_dataset` must be specified.

### modelTesting

In [51]:
try:
    interpreter_float16 = tf.lite.Interpreter(TFLITE_FILE_PATH_FLOAT16)
    signature_float16 = interpreter_float16.get_signature_runner()
except Exception as e:
    print(e)
    
try: 
    interpreter_float32 = tf.lite.Interpreter(TFLITE_FILE_PATH_FLOAT32)
    signature_float32 = interpreter_float32.get_signature_runner()
except Exception as e:
    print(e)

try: 
    interpreter_int8 = tf.lite.Interpreter(TFLITE_FILE_PATH_INT8)
    signature_int8 = interpreter_int8.get_signature_runner()
except Exception as e:
    print(e)


Could not open './models/tflite_quant_model_int8.tflite'.


In [48]:
TEST_IMG_PATH = """data/test_img/test.jpg"""
def prepareImage(path):
    img = tf.keras.utils.load_img(path)
    img = tf.keras.utils.normalize(img, axis=1)
    img = tf.cast(img, tf.uint8)
    shape = (1) + img.shape
    img = tf.reshape(img, shape)
    return img
img = prepareImage(TEST_IMG_PATH)


In [89]:
import os

EVAL_IMAGE_PATH = """./data/val2017/"""
prep_imgs = []
for path, subdirs, files in os.walk(EVAL_IMAGE_PATH):
    for name in files[0:1000]:
        prep_imgs.append(prepareImage(EVAL_IMAGE_PATH + name))

results = []
for prep_img in prep_imgs:
    results.append(model(prep_img))


In [49]:
import os

EVAL_IMAGE_PATH = """./data/val2017/"""
prep_imgs = []
for path, subdirs, files in os.walk(EVAL_IMAGE_PATH):
    for name in files[0:1000]:
        prep_imgs.append(prepareImage(EVAL_IMAGE_PATH + name))

results = []
for prep_img in prep_imgs:
    results.append(signature_float16(prep_img))


TypeError: __call__() takes 1 positional argument but 2 were given

In [57]:
interpreter_float16.allocate_tensors()

AttributeError: 'bytes' object has no attribute 'allocate_tensors'

## Creating a dataset

In [5]:
# !wget http://images.cocodataset.org/zips/val2017.zip
# !unzip -q val2017.zip

--2022-11-22 15:24:08--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.84.0, 52.217.8.212, 52.216.164.107, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.84.0|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: 'val2017.zip'

     0K .......... .......... .......... .......... ..........  0%  204K 65m5s
    50K .......... .......... .......... .......... ..........  0%  714K 41m50s
   100K .......... .......... .......... .......... ..........  0%  804K 33m23s
   150K .......... .......... .......... .......... ..........  0% 8,14M 25m26s
   200K .......... .......... .......... .......... ..........  0%  447K 26m17s
   250K .......... .......... .......... .......... ..........  0% 8,16M 22m10s
   300K .......... .......... .......... .......... ..........  0% 8,44M 19m13s
   350K .......... .......... .......... .......... .

In [ ]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 244, 244, 3)
      yield [data.astype(np.float32)]

## print stats

In [ ]:
import matplotlib.pyplot as plt

stats = {
    'size': {
        'float32': 24.942224,
        'float16': 12.857228,
        'int8': 0,
    },
    'accuracy': {
        'float32': 0,
        'float16': 0,
        'int8': 0,
    },
    'latency': {
        'float32': 0,
        'float16': 0,
        'int8': 0,
    }
}

def printDict(dict, title="", ylabel="", show=False):
    plt.bar(range(len(dict)), list(dict.values()), align='center')
    plt.xticks(range(len(dict)), list(dict.keys()))
    plt.title(title)
    plt.ylabel(ylabel)
    if show:
        plt.show()

printDict(stats["size"], title="size", ylabel="size (MB)", show=True)
printDict(stats["accuracy"], title="accuracy", ylabel="accuracy (%)", show=True)
printDict(stats["latency"], title="latency", ylabel="latency (sec)", show=True)